In [1]:
import pandas as pd

from IPython.display import HTML

In [38]:
dh = pd.read_csv('sequencias.csv')

In [39]:
dh

,demanda,sequencia
0,38,"(('DESCONHECIDO', 'Estimar pontos de função'),..."
1,73,"(('DESCONHECIDO', 'Estimar pontos de função'),..."
2,74,"(('DESCONHECIDO', 'Estimar pontos de função'),..."
3,77,"(('Realizar implementação', 'Elaborar procedim..."
4,78,"(('Realizar implementação', 'Elaborar procedim..."
...,...,...
24733,25498,"(('Novo', 'Desenvolver internamente'), ('Desen..."
24734,25500,"(('Novo', 'Desenvolver internamente'), ('Desen..."
24735,25504,"(('Novo', 'Desenvolver internamente'), ('Desen..."
24736,25507,"(('Novo', 'Demanda cancelada'))"


In [40]:
dh.loc[dh.demanda==38, 'sequencia'].values[0]

"(('DESCONHECIDO', 'Estimar pontos de função'), ('Estimar pontos de função', 'Elaborar parecer: desenvolver ou adquirir?'), ('Elaborar parecer: desenvolver ou adquirir?', 'Considerar decisão da DISIS: ratificar ou retificar'), ('Considerar decisão da DISIS: ratificar ou retificar', 'Atualizar registro da demanda'), ('Atualizar registro da demanda', 'Demanda sem ônus em pontos de função concluída'))"

In [41]:
dh.nunique()

demanda      24738
sequencia      598
dtype: int64

In [131]:
dt = dh.groupby(['sequencia']).nunique().sort_values(by=['demanda'], ascending=False)

dt.to_csv('fluxos.csv')

In [156]:
dt1 = pd.read_csv('fluxos.csv')
dt1.sequencia.map(str)

0      (('Novo', 'Em andamento'), ('Em andamento', 'C...
1      (('Novo', 'Desenvolver internamente'), ('Desen...
2      (('Novo', 'Demanda sem ônus em pontos de funçã...
3      (('Novo', 'Desenvolver internamente'), ('Desen...
4      (('Novo', 'Preparar documentação para fábrica'...
                             ...                        
593    (('Novo', 'Desenvolver internamente'), ('Desen...
594    (('Novo', 'Desenvolver internamente'), ('Desen...
595    (('Novo', 'Desenvolver internamente'), ('Desen...
596    (('Novo', 'Desenvolver internamente'), ('Desen...
597    (('Realizar implementação', 'Testar implementa...
Name: sequencia, Length: 598, dtype: object

In [164]:
# quanto ao ônus

dt2 = dt1.copy()

com_onus = dt2.sequencia.str.endswith(("Demanda com ônus em pontos de função concluída'))",
                                         "Demanda concluída com ônus'))"))
sem_onus = dt2.sequencia.str.endswith(("Demanda sem ônus em pontos de função concluída'))", 
                                         "Demanda concluída sem ônus'))"))

dt2.loc[com_onus, 'onus'] = 'com'
dt2.loc[sem_onus, 'onus'] = 'sem'

# quanto a finalizacao em geral

eh_cancelada = dt2.sequencia.str.endswith("Demanda cancelada'))")
eh_dba = dt2.sequencia.str.endswith(( "Concluído'))", "Aceito'))", "Cancelado'))" ))
eh_defeito_ou_pdti = dt2.sequencia.str.endswith(( "Fechado'))", "'Iniciado'))", "Cancelado'))", "Rejeição cancelada'))" ))

dt2.loc[eh_cancelada, 'onus'] = 'cancelada'
dt2.loc[eh_dba, 'onus'] = 'dba'
dt2.loc[eh_defeito_ou_pdti, 'onus'] = 'defeito_ou_pdti'

In [168]:
eh_fabrica = dt2.sequencia.str.contains("fábrica") | dt2.sequencia.str.contains("artefato") | dt2.sequencia.str.contains("Contar") & (~dt2.sequencia.str.contains("iternamente"))
#(
#    dt2.sequencia.str.contains("Realizar implementação") 
#    | dt2.sequencia.str.contains("Executar manutenção") 
#    | dt2.sequencia.str.contains("Contar pontos de função")
#    & (~dt2.sequencia.str.contains("Desenvolver internamente"))
#)
eh_interna = dt2.sequencia.str.contains("internamente") & (~dt2.sequencia.str.contains("fábrica") & ~dt2.sequencia.str.contains("artefato")) & ~dt2.sequencia.str.contains("Contar")
#(
#    dt2.sequencia.str.contains("Desenvolver internamente") 
#    & ~dt2.sequencia.str.contains("Realizar implementação") 
#    & ~dt2.sequencia.str.contains("Executar manutenção")
#)

#dt2['tipo'] = 'aberta'  ## default 

dt2.loc[eh_fabrica, 'tipo'] = 'fabrica'
dt2.loc[eh_interna, 'tipo'] = 'interna'

In [80]:
dt3 = dt2.loc[dt2.tipo.isna()]
HTML(dt3.to_html())

,sequencia,demanda,sequencia.1,onus,tipo


In [169]:
dt2.to_csv('fluxos_classificados.csv')

In [109]:
dt2.loc[(dt2.onus=='com') & (dt2.tipo=='fabrica')].count()

sequencia      309
demanda        309
sequencia.1    309
onus           309
tipo           309
dtype: int64

In [82]:
HTML(dt2.to_html())

,sequencia,demanda,sequencia.1,onus,tipo
0,"(('Novo', 'Em andamento'), ('Em andamento', 'Concluído'))",11844,1,NaN,dba
1,"(('Novo', 'Desenvolver internamente'), ('Desenvolver internamente', 'Demanda sem ônus em pontos de função concluída'))",1824,1,sem,interna
2,"(('Novo', 'Demanda sem ônus em pontos de função concluída'))",1445,1,sem,interna
3,"(('Novo', 'Desenvolver internamente'), ('Desenvolver internamente', 'Demanda concluída sem ônus'))",1249,1,sem,interna
4,"(('Novo', 'Preparar documentação para fábrica'), ('Preparar documentação para fábrica', 'Pronto para fábrica'), ('Pronto para fábrica', 'Realizar implementação'), ('Realizar implementação', 'Testar implementação'), ('Testar implementação', 'Homologar com Demandante'), ('Homologar com Demandante', 'Entregar artefatos'), ('Entregar artefatos', 'Validar artefatos'), ('Validar artefatos', 'Contar pontos de função'), ('Contar pontos de função', 'Atualizar registro da demanda'), ('Atualizar registro da demanda', 'Demanda com ônus em pontos de função concluída'))",1190,1,com,fabrica
5,"(('Novo', 'Em andamento'), ('Em andamento', 'Concluído'), ('Concluído', 'Aceito'))",1006,1,NaN,dba
6,"(('Novo', 'Demanda cancelada'))",598,1,NaN,cancelada
7,"(('Novo', 'Desenvolver internamente'), ('Desenvolver internamente', 'Validar com a Área Demandante'), ('Validar com a Área Demandante', 'Demanda sem ônus em pontos de função concluída'))",534,1,sem,interna
8,"(('Preparar documentação para fábrica', 'Atualizar registro da demanda'), ('Atualizar registro da demanda', 'Demanda sem ônus em pontos de função concluída'))",451,1,sem,interna
9,"(('Novo', 'Realizar orçamento'), ('Realizar orçamento', 'Validar orçamento'), ('Validar orçamento', 'Especificar testes'), ('Especificar testes', 'Validar especificação/testes'), ('Validar especificação/testes', 'Realizar implementação'), ('Realizar implementação', 'Testar implementação'), ('Testar implementação', 'Homologar com Demandante'), ('Homologar com Demandante', 'Entregar artefatos'), ('Entregar artefatos', 'Validar artefatos'), ('Validar artefatos', 'Atualizar registro da demanda'), ('Atualizar registro da demanda', 'Demanda concluída com ônus'))",426,1,com,fabrica
